In [ ]:
import numpy as np
import cadquery as cq
from microgen import *

from jupyter_cadquery import show, set_defaults, open_viewer, Part
from jupyter_cadquery.replay import enable_replay, disable_replay, reset_replay, get_context, replay, Replay, _CTX
from jupyter_cadquery.cad_objects import to_assembly

cv = open_viewer("Box", cad_width=640, height=480)

set_defaults(reset_camera=True, show_parent=False, axes=True, axes0=True)

# Shapes available

In [ ]:
shapes = {"box":             {"dim": (0.8, 0.8, 0.8)},
          "sphere":          {"radius": 0.5},
          "capsule":         {"height": 0.5, "radius": 0.3},
          "cylinder":        {"height": 0.5, "radius": 0.5},
          "ellipsoid":       {"radii": (0.5, 0.25, 0.3)},
          "extrudedpolygon": {"listCorners": [(0.5, 0), (0.25, 0.44), (-0.25, 0.44), 
                                              (-0.5, 0), (-0.25, -0.44), (0.25, -0.44), (0.5, 0)],
                              "height": 0.5}} #, "polyhedron":      {"dic": }}


assembly = cq.Assembly(name="Assembly")

n_col = 3
n_row = np.ceil(len(shapes)/n_col)
i = 0
for shape, param_geom in shapes.items():
    i_x = i%n_col
    i_y = i//n_col
    elem = newGeometry(
        shape=shape,
        center=(1.2 * (i_x - 0.5 * (n_col - 1)), -1.2 * (i_y - 0.5 * (n_row - 1)), 0),
        orientation=(90, 90, 90),
        param_geom=param_geom,
    )
    assembly.add(elem.generate(), name=elem.shape)
    i = i + 1

show(assembly)

# Honeycomb

In [ ]:
from microgen import Box, ExtrudedPolygon, Phase, cutPhaseByShapeList, mesh
import numpy as np
import cadquery as cq

side_length = 2.5  # side in mm of the hexagon
poly_height = 2.5  # height in mm of the hexagon
theta = 30 * np.pi / 180  # half angle of the hexagone

h0 = 0.5 * poly_height
h1 = np.cos(theta) * side_length
h2 = abs(np.sin(theta) * side_length)

thickness = 30  # mm

with open('seedList.data', 'r') as f:
    seedList = [[1, 1, 1]]
    seedList = np.genfromtxt(f, delimiter="\t")

box = Box(dim=(thickness, 60, 60))

shapeList = []
for seed in seedList:
    poly = ExtrudedPolygon(
        center=(seed[0] - thickness, seed[1], seed[2]),
        listCorners=[
            (0, h2 + h0),
            (h1, h0),
            (h1, -h0),
            (0, -h2 - h0),
            (-h1, -h0),
            (-h1, h0),
            (0, h2 + h0),
        ],
        height=thickness,
    )
    shapeList.append(poly.generate())

boxPhase = Phase(shape=box.generate())

honeycomb = cutPhaseByShapeList(phaseToCut=boxPhase, cqShapeList=shapeList)
show(honeycomb.shape)

# Octet-Truss

In [ ]:
import os
import numpy as np
import cadquery as cq
from microgen import (
    Rve,
    Cylinder,
    periodic,
    cutPhases,
    meshPeriodic,
    Phase,
)

# ----------LOADTXT------------------------------------------------------------------------------------------#

dir = os.path.dirname(os.path.realpath("__file__"))
# path
path_data = dir + "/"
Ngeomphase_file = "test_octet.dat"

# fichier
NPhases_file = path_data + Ngeomphase_file

dt = np.dtype(
    [
        ("number", int),
        ("shape", np.str_, 10),
        ("xc", np.float64),
        ("yc", np.float64),
        ("zc", np.float64),
        ("psi", np.float64),
        ("theta", np.float64),
        ("phi", np.float64),
        ("a1", np.float64),
        ("a2", np.float64),
    ]
)
# précision du type des données
DATA = np.loadtxt(
    NPhases_file,
    dtype=dt,
    usecols=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9),
    skiprows=1,
    unpack=True,
    ndmin=1,
)

xc = DATA[2]
yc = DATA[3]
zc = DATA[4]
psi = DATA[5]
theta = DATA[6]
phi = DATA[7]
height = DATA[8]
radius = DATA[9]

# sections = read_sections(path_data,section_file)

rve = Rve(dim=1)
listPhases = []
listPeriodicPhases = []
n = len(xc)

for i in range(0, n):
    elem = Cylinder(
        center=(xc[i] - 0.5, yc[i] - 0.5, zc[i] - 0.5),
        orientation=(psi[i], theta[i], phi[i]),
        height=height[i],
        radius=radius[i],
    )
    listPhases.append(Phase(shape=elem.generate()))

for phase_elem in listPhases:
    periodicPhase = periodic(phase=phase_elem, rve=rve)
    listPeriodicPhases.append(periodicPhase)

phases_cut = cutPhases(
    phaseList=listPeriodicPhases, reverseOrder=False
)
octet_truss = cq.Compound.makeCompound([phase.shape for phase in phases_cut])
show(Part(octet_truss))

# Periodic geometry

In [ ]:
rve = Rve(dim=1)
final_geom = repeatShape(octet_truss, rve, grid=[3, 3, 3])

show(final_geom)

# Gyroid TPMS

In [ ]:
from microgen import Tpms, surface_functions
import cadquery as cq

geometry = Tpms(
    surface_function=surface_functions.gyroid,
    offset=0.3,
)
shape = geometry.generate(type_part="sheet")
show(shape)

# Raster shape

In [ ]:
import cadquery as cq
from microgen import Rve, Ellipsoid, mesh, Phase, rasterPhase

rve = Rve(dim=1)

elem = Ellipsoid(radii=(0.15, 0.31, 0.4))
elli = elem.generate()

raster = rasterPhase(phase=Phase(shape=elli), rve=rve, grid=[5, 5, 5])

compound = cq.Compound.makeCompound([solid for phase in raster for solid in phase.solids])
show(compound)